In [102]:
from __future__ import print_function
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

# 參數設定
batch_size = 64  # Batch size for training.
epochs = 2  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# 設定訓練資料檔路徑
#data_path = 'fra-eng/fra.txt'
data_path = 'cmn.txt' 

# 讀取資料檔，並將所有單字整理為字典，分別為英文及中文字典，注意，英文為字母的集合，非單字(Word)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, encoding ='utf8').read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text = line.split('\t')
    target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    #target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [103]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# 計算編碼器、解碼器的最大長度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# 以dict儲存字典單字及序號
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

Number of samples: 10000
Number of unique input tokens: 28396
Number of unique output tokens: 28396
Max sequence length for inputs: 3
Max sequence length for outputs: 3


In [104]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# 建立 encoder LSTM 隱藏層
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 定義模型，由 encoder_input_data 及 decoder_input_data 轉換為 decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [105]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
          
          
# 儲存模型及結果
model.save('s2s.h5')

Epoch 1/2
125/125 [==============================] - 79s 616ms/step - loss: 7.0147 - val_loss: 7.5779
Epoch 2/2
125/125 [==============================] - 76s 606ms/step - loss: 6.7649 - val_loss: 8.1642


In [28]:
encoder_model = Model(encoder_inputs, encoder_states)

# 定義解碼器的input
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 定義解碼器 LSTM 模型
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
    
# 以編碼器的記憶狀態 h 及 c 為解碼器的記憶狀態  
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# 建立反向的 dict，才能透過查詢將數值轉回文字
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [70]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 1] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [101]:
from opencc import OpenCC
data_change = OpenCC('s2twp')
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('*')
    print( input_texts[seq_index])
    try:
        
        print('Decoded sentence:', data_change.convert.decoded_sentence)
    except:
        # 出現亂碼，以?取代
        
        print('Decoded sentence:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])

ValueError: in user code:

    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1461 run_step  **
  

In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [8]:
from tensorflow import keras

In [6]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\eager\context.py)

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'cmn.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text  = line.split('\t')
    target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    #target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 28396
Number of unique output tokens: 28396
Max sequence length for inputs: 3
Max sequence length for outputs: 3


In [82]:
from opencc import OpenCC
data_change = OpenCC('s2twp')
with open('cmn.txt', 'r', encoding='utf-8') as f:
    data = f.read()
data = data.split('\n')
data = data[:100]
print(data[-5:])

['Good job!\t干的好！\tCC-BY 2.0 (France) Attribution: tatoeba.org #1021038 (Quazel) & #7767580 (jiangche)', 'Grab Tom.\t抓住汤姆。\tCC-BY 2.0 (France) Attribution: tatoeba.org #1972639 (CK) & #5092688 (mirrorvan)', 'Grab him.\t抓住他。\tCC-BY 2.0 (France) Attribution: tatoeba.org #285184 (CK) & #5092265 (mirrorvan)', 'Have fun.\t玩得開心。\tCC-BY 2.0 (France) Attribution: tatoeba.org #21267 (CK) & #819292 (Martha)', 'He tries.\t他来试试。\tCC-BY 2.0 (France) Attribution: tatoeba.org #288166 (CM) & #5092267 (mirrorvan)']


In [83]:
en_data = [line.split('\t')[0] for line in data]
ch_data = ['\t' + line.split('\t')[1] + '\n' for line in data]
print('英文數據:\n', en_data[:10])
print('\n中文數據:\n', ch_data[:10])

# 分別生成中英文字典
en_vocab = set(''.join(en_data))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(ch_data))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print('\n英文字典:\n', en2id)
print('\n中文字典共計\n:', ch2id)

英文數據:
 ['Hi.', 'Hi.', 'Run.', 'Wait!', 'Wait!', 'Begin.', 'Hello!', 'I try.', 'I won!', 'Oh no!']

中文數據:
 ['\t嗨。\n', '\t你好。\n', '\t你用跑的。\n', '\t等等！\n', '\t等一下！\n', '\t开始！\n', '\t你好。\n', '\t我试试。\n', '\t我赢了。\n', '\t不会吧。\n']

英文字典:
 {'T': 0, 'J': 1, 'i': 2, ' ': 3, 'p': 4, 'G': 5, 'm': 6, 's': 7, 'A': 8, 'j': 9, 'I': 10, '.': 11, 't': 12, 'd': 13, 'N': 14, 'W': 15, 'O': 16, 'H': 17, 'n': 18, 'R': 19, 'L': 20, "'": 21, 'o': 22, 'g': 23, 'v': 24, 'q': 25, 'C': 26, 'l': 27, 'r': 28, 'B': 29, 'k': 30, 'w': 31, 'Y': 32, 'K': 33, 'a': 34, 'h': 35, 'y': 36, 'c': 37, 'f': 38, 'D': 39, 'P': 40, '?': 41, 'b': 42, '!': 43, 'S': 44, 'e': 45, 'u': 46}

中文字典共計
: {'进': 0, '用': 1, '病': 2, '点': 3, '门': 4, '善': 5, '，': 6, '经': 7, '平': 8, '友': 9, '完': 10, '确': 11, '帮': 12, '拿': 13, '公': 14, '已': 15, '走': 16, '他': 17, '閉': 18, '乾': 19, '吻': 20, '静': 21, '请': 22, '抱': 23, '跳': 24, '好': 25, '诉': 26, '再': 27, '管': 28, '做': 29, '開': 30, '留': 31, '冷': 32, '滚': 33, '家': 34, '趴': 35, '听': 36, '什': 37, '汤': 38, '得':

In [84]:
en_num_data = [[en2id[en] for en in line ] for line in en_data]
ch_num_data = [[ch2id[ch] for ch in line] for line in ch_data]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in ch_data]

print('char:', en_data[1])
print('index:', en_num_data[1])

char: Hi.
index: [17, 2, 11]


In [85]:
import numpy as np

# 獲取輸入輸出端的最大長度
max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.
    for t, j in enumerate(de_num_data[i]):
        decoder_target_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

max encoder length: 9
max decoder length: 9
index data:
 [17, 2, 11]
one hot data:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [160]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.001
BATCH_SIZE = 10
EPOCHS = 1000

In [161]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np

# ==============encoder=============
encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
#emb_inp = Embedding(output_dim=HIDDEN_SIZE, input_dim=EN_VOCAB_SIZE)(encoder_inputs)
encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)

In [162]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))

#emb_target = Embedding(output_dim=HIDDEN_SIZE, input_dim=CH_VOCAB_SIZE, mask_zero=True)(decoder_inputs)
lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
decoder_outputs = decoder_dense(decoder_h2)

In [163]:
decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))

#emb_target = Embedding(output_dim=HIDDEN_SIZE, input_dim=CH_VOCAB_SIZE, mask_zero=True)(decoder_inputs)
lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')

decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
decoder_outputs = decoder_dense(decoder_h2)

In [164]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.)

Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_56 (InputLayer)           [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
input_58 (InputLayer)           [(None, None, 403)]  0                                            
__________________________________________________________________________________________________
lstm_60 (LSTM)                  [(None, None, 256),  330752      input_56[0][0]                   
__________________________________________________________________________________________________
lstm_64 (LSTM)                  [(None, None, 256),  675840      input_58[0][0]                   
                                                                 lstm_60[0][1]             

10/10 [==============================] - 1s 79ms/step - loss: 2.1379 - accuracy: 0.3877
Epoch 60/1000
10/10 [==============================] - 1s 79ms/step - loss: 2.0320 - accuracy: 0.4090
Epoch 61/1000
10/10 [==============================] - 1s 77ms/step - loss: 1.9571 - accuracy: 0.4461
Epoch 62/1000
10/10 [==============================] - 1s 70ms/step - loss: 1.8558 - accuracy: 0.4273
Epoch 63/1000
10/10 [==============================] - 1s 70ms/step - loss: 1.9114 - accuracy: 0.4474
Epoch 64/1000
10/10 [==============================] - 1s 70ms/step - loss: 1.7913 - accuracy: 0.4642
Epoch 65/1000
10/10 [==============================] - 1s 71ms/step - loss: 1.8035 - accuracy: 0.4714
Epoch 66/1000
10/10 [==============================] - 1s 72ms/step - loss: 1.7263 - accuracy: 0.4882
Epoch 67/1000
10/10 [==============================] - 1s 73ms/step - loss: 1.6550 - accuracy: 0.4770
Epoch 68/1000
10/10 [==============================] - 1s 70ms/step - loss: 1.6399 - accuracy: 0

10/10 [==============================] - 1s 70ms/step - loss: 0.3510 - accuracy: 0.7169
Epoch 140/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.3456 - accuracy: 0.7322
Epoch 141/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.3316 - accuracy: 0.7173
Epoch 142/1000
10/10 [==============================] - 1s 69ms/step - loss: 0.3245 - accuracy: 0.7306
Epoch 143/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.3256 - accuracy: 0.7449
Epoch 144/1000
10/10 [==============================] - 1s 74ms/step - loss: 0.3559 - accuracy: 0.7128
Epoch 145/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.3353 - accuracy: 0.7143
Epoch 146/1000
10/10 [==============================] - 1s 73ms/step - loss: 0.3831 - accuracy: 0.7233
Epoch 147/1000
10/10 [==============================] - 1s 74ms/step - loss: 0.5234 - accuracy: 0.6914
Epoch 148/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.6348 - ac

10/10 [==============================] - 1s 70ms/step - loss: 0.0857 - accuracy: 0.7816
Epoch 219/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.0907 - accuracy: 0.7729
Epoch 220/1000
10/10 [==============================] - 1s 69ms/step - loss: 0.1043 - accuracy: 0.7868
Epoch 221/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.0789 - accuracy: 0.7835
Epoch 222/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.0820 - accuracy: 0.8056
Epoch 223/1000
10/10 [==============================] - 1s 74ms/step - loss: 0.1482 - accuracy: 0.7875
Epoch 224/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.2525 - accuracy: 0.7366
Epoch 225/1000
10/10 [==============================] - 1s 69ms/step - loss: 0.2678 - accuracy: 0.7388
Epoch 226/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.2438 - accuracy: 0.7237
Epoch 227/1000
10/10 [==============================] - 1s 69ms/step - loss: 0.2054 - ac

10/10 [==============================] - 1s 71ms/step - loss: 0.0170 - accuracy: 0.8001
Epoch 298/1000
10/10 [==============================] - 1s 73ms/step - loss: 0.0160 - accuracy: 0.8048
Epoch 299/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0156 - accuracy: 0.8015
Epoch 300/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0157 - accuracy: 0.8132
Epoch 301/1000
10/10 [==============================] - 1s 73ms/step - loss: 0.0158 - accuracy: 0.7892
Epoch 302/1000
10/10 [==============================] - 1s 73ms/step - loss: 0.0158 - accuracy: 0.7882
Epoch 303/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0163 - accuracy: 0.7877
Epoch 304/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0159 - accuracy: 0.7918
Epoch 305/1000
10/10 [==============================] - 1s 77ms/step - loss: 0.0154 - accuracy: 0.7890
Epoch 306/1000
10/10 [==============================] - 1s 80ms/step - loss: 0.0158 - ac

...

10/10 [==============================] - 1s 73ms/step - loss: 0.0081 - accuracy: 0.7781
Epoch 614/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0082 - accuracy: 0.7939
Epoch 615/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0079 - accuracy: 0.7962
Epoch 616/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0077 - accuracy: 0.7912
Epoch 617/1000
10/10 [==============================] - 1s 76ms/step - loss: 0.0080 - accuracy: 0.8018
Epoch 618/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0078 - accuracy: 0.8001
Epoch 619/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0079 - accuracy: 0.7805
Epoch 620/1000
10/10 [==============================] - 1s 74ms/step - loss: 0.0078 - accuracy: 0.8009
Epoch 621/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0076 - accuracy: 0.8066
Epoch 622/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0075 - ac

10/10 [==============================] - 1s 80ms/step - loss: 0.0056 - accuracy: 0.7924
Epoch 693/1000
10/10 [==============================] - 1s 82ms/step - loss: 0.0055 - accuracy: 0.8053
Epoch 694/1000
10/10 [==============================] - 1s 79ms/step - loss: 0.0055 - accuracy: 0.7929
Epoch 695/1000
10/10 [==============================] - 1s 83ms/step - loss: 0.0054 - accuracy: 0.7994
Epoch 696/1000
10/10 [==============================] - 1s 79ms/step - loss: 0.0058 - accuracy: 0.7724
Epoch 697/1000
10/10 [==============================] - 1s 78ms/step - loss: 0.0055 - accuracy: 0.7848
Epoch 698/1000
10/10 [==============================] - 1s 75ms/step - loss: 0.0054 - accuracy: 0.7905
Epoch 699/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.0053 - accuracy: 0.8038
Epoch 700/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0054 - accuracy: 0.7909
Epoch 701/1000
10/10 [==============================] - 1s 76ms/step - loss: 0.0052 - ac

10/10 [==============================] - 1s 69ms/step - loss: 0.0044 - accuracy: 0.7878
Epoch 772/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0042 - accuracy: 0.7865
Epoch 773/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0044 - accuracy: 0.7864
Epoch 774/1000
10/10 [==============================] - 1s 69ms/step - loss: 0.0043 - accuracy: 0.8039
Epoch 775/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0043 - accuracy: 0.7923
Epoch 776/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0044 - accuracy: 0.7938
Epoch 777/1000
10/10 [==============================] - 1s 69ms/step - loss: 0.0043 - accuracy: 0.7902
Epoch 778/1000
10/10 [==============================] - 1s 69ms/step - loss: 0.0042 - accuracy: 0.7943
Epoch 779/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0041 - accuracy: 0.7870
Epoch 780/1000
10/10 [==============================] - 1s 73ms/step - loss: 0.0041 - ac

10/10 [==============================] - 1s 79ms/step - loss: 0.0035 - accuracy: 0.8024
Epoch 851/1000
10/10 [==============================] - 1s 79ms/step - loss: 0.0034 - accuracy: 0.7754
Epoch 852/1000
10/10 [==============================] - 1s 77ms/step - loss: 0.0035 - accuracy: 0.8026
Epoch 853/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.0034 - accuracy: 0.7991
Epoch 854/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.0037 - accuracy: 0.8000
Epoch 855/1000
10/10 [==============================] - 1s 74ms/step - loss: 0.0034 - accuracy: 0.8040
Epoch 856/1000
10/10 [==============================] - 1s 79ms/step - loss: 0.0034 - accuracy: 0.7987
Epoch 857/1000
10/10 [==============================] - 1s 75ms/step - loss: 0.0034 - accuracy: 0.7961
Epoch 858/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0034 - accuracy: 0.7942
Epoch 859/1000
10/10 [==============================] - 1s 73ms/step - loss: 0.0035 - ac

10/10 [==============================] - 1s 72ms/step - loss: 0.0118 - accuracy: 0.7984
Epoch 930/1000
10/10 [==============================] - 1s 72ms/step - loss: 0.0112 - accuracy: 0.7867
Epoch 931/1000
10/10 [==============================] - 1s 70ms/step - loss: 0.0115 - accuracy: 0.7962
Epoch 932/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0106 - accuracy: 0.8053
Epoch 933/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0097 - accuracy: 0.7901
Epoch 934/1000
10/10 [==============================] - 1s 71ms/step - loss: 0.0091 - accuracy: 0.7874
Epoch 935/1000
10/10 [==============================] - 1s 80ms/step - loss: 0.0089 - accuracy: 0.7896
Epoch 936/1000
10/10 [==============================] - 1s 79ms/step - loss: 0.0087 - accuracy: 0.7987
Epoch 937/1000
10/10 [==============================] - 1s 83ms/step - loss: 0.0090 - accuracy: 0.7966
Epoch 938/1000
10/10 [==============================] - 1s 75ms/step - loss: 0.0086 - ac

In [165]:
encoder_model = Model(encoder_inputs, [encoder_state_h1, encoder_state_c1, encoder_state_h2, encoder_state_c2])

# 預測模型中的decoder的初始化狀態需要傳入新的狀態
decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))

# 使用傳入的值來初始化當前模型的輸入狀態
decoder_h1, state_h1, state_c1 = lstm1(decoder_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1])
decoder_h2, state_h2, state_c2 = lstm2(decoder_h1, initial_state=[decoder_state_input_h2, decoder_state_input_c2])
decoder_outputs = decoder_dense(decoder_h2)

decoder_model = Model([decoder_inputs, decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2], 
                      [decoder_outputs, state_h1, state_c1, state_h2, state_c2])

In [167]:
for k in range(40,50):
    test_data = encoder_input_data[k:k+1]
    h1, c1, h2, c2 = encoder_model.predict(test_data)
    target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
    target_seq[0, 0, ch2id['@']] = 1
    outputs = []
    while True:
        output_tokens, h1, c1, h2, c2 = decoder_model.predict([target_seq, h1, c1, h2, c2])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        outputs.append(sampled_token_index)
        target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
        target_seq[0, 0, sampled_token_index] = 1
        if sampled_token_index == ch2id['。'] or len(outputs) > 20: break
    
    print(en_data[k])
    print(''.join([id2ch[i] for i in outputs]))

Go away!
海伦：冰岛有温泉？。
Go away!
这是几杆洞？。
Go away.
生产者蜡丸戴维斯。
Go home.
拥抱你的孩子。
Goodbye!
里面有上千个托座。
Goodbye!
交易由管理员批准。
Hang on!
嗯，这并不太妙。
Hang on!
雪貂会不会生跳蚤？。
Hang on.
F：在海牙。
He came.
没有定冠词。


In [106]:
import json
from tensorflow.keras import optimizers
import numpy as np

data = []
endata = []
chdata = []

def chine(string):
    for ch in string :
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

for line in open('train.json', 'r', encoding='utf-8'):
    data.append(json.loads(line))  


print('----------')

for i in range(len(data)):
    if chine(data[i]['chinese']) == True :
        if len(data[i]['chinese']) < 10:
            endata.append(data[i]['english'])
            chdata.append('@'+data[i]['chinese']+'。')
        if len(chdata) == 100 :
            break

en_vocab = set(''.join(endata))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(chdata))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print('英文數據:\n', endata)
print('\n中文數據:\n', chdata)
# print('\n英文字典:\n', en2id)
# print('\n中文字典共計\n:', ch2id)


# en_vocab = set(''.join(endata))
# id2en = list(en_vocab)
# en2id = {c:i for i,c in enumerate(id2en)}

----------
英文數據:
 ['Look at these coasters over here.', 'Choose a recorder.', "I hadn't paid the telephone bill.", "That's easier said than done, of course.", 'Side-to-Side Movements.', 'about like 80 degrees.', 'We all are from Shandong.', 'She was possessed by a devil.', 'This wool knits up well.', 'The majority was wrong last time.', 'Stone Soup Stories to Go!', 'Done. See you tomorrow.', 'He eased some of the strains on the poor.', 'Could it be that it was written wrongly?', 'What a terrible temper!', 'Great talents flower late.', 'I forbid you to make a sortie today.', 'C：My surname is Jiang.', 'Well, if it was greater .', 'They looked over to the left.', 'To supervise the management of printing industry.', 'no one else can see you shake your head.', 'All photos dials.', 'Stained glass window panels;', 'The murderer was caught red-handed.', 'You don’t love Melanie.', 'His style is very lucid .', 'At this time, there was a male cat.', 'The truth has leaked out.', 'Material evidence

In [107]:
en_num_data = [[en2id[en] for en in line ] for line in endata]
ch_num_data = [[ch2id[ch] for ch in line] for line in chdata]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in chdata]

print('char:', endata[1])
print('index:', en_num_data[1])
print(len(en_num_data))
len(ch_num_data)
# de_num_data

char: Choose a recorder.
index: [35, 46, 28, 28, 10, 64, 3, 47, 3, 37, 64, 50, 28, 37, 19, 64, 37, 14]
100


100

In [108]:
import numpy as np

max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.
    for t, j in enumerate(de_num_data[i]):
        decoder_target_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

max encoder length: 49
max decoder length: 11
index data:
 [35, 46, 28, 28, 10, 64, 3, 47, 3, 37, 64, 50, 28, 37, 19, 64, 37, 14]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [109]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.01
BATCH_SIZE = 20
EPOCHS = 250